# Introduction

인공지능으로 언어를 생성하는 것은 어려운 일입니다.

최근의 비주얼 도메인에서의 강력한 생성 모델과 비교면 성능이후집니다.

이 논문은 그럴듯한 텍스트 문장을 생성하는데 목적을두고있다.

이 논문의 언어 생성모델은 문장의 잠재변수들을 효과적으로 파악하면서

그 잠재변수중 우리가 원하는 의미론적 정보를 다루고자 한다.

우리는 생성 문장에 효과적으로 의미를 표현하기 위해

VAE와 Discriminator를 결합한 새로운 신경 생성 모델을 제안한다.

![](./img/1.png)

이 모델은 가짜 샘플을 여분의 훈련 데이터로 활용하기 위해

웨이크 슬립 알고리즘을 사용해 VAE를 향상시키는 것으로 볼 수 있습니다.

![](./img/2.png)

이산 공간의 텍스트 샘플에 대해 미분 가능한 공간에서의 근사를 수행합니다.

의미론적 속성을 컨트롤하는 과정을

Generator와 Discriminator의 효율적인 공동 학습을 통해 달성합니다.

최종적으로 이 모델은 감정 및 시제 속성을 가진 문장을 생성합니다.

Discriminator를 평가자로 사용하여 생성된 문장의 정확성을 검증합니다.

# Controlled Generation of Text

우리의 모델은 지정된 의미론적 구조가 부여 된 표현 벡터를 조건으로하는

그럴듯한 문장을 생성하는 것을 목표로한다.

### 긍부정 예시

예를 들어, 특정 의미론적 구조의 벡터를 컨트롤 하기 위해

우리 모델은 잠재변수의 한 차원을 "긍정"와 "부정"의미를 인코딩하도록 할당하고

단순히 특정 코드(긍정이나 부정)를 지정하여 원하는 의미를 가진 샘플을 생성합니다.

![](./img/3.png)

의미론적 벡터를 따로 분해한 구조로부터

우리는 다른 기능과 독립적인 각각의 의미론적 코드의

두드러진 속성을 포착 할 수 있습니다.

우리 모델의 이점은 다음과 같습니다.

1) Discriminator가 Generator를 도와 효과적으로 의도된 의미가 부여된 텍스트 생성

2) 잠재변수로부터 원하는 속성을 추출해 모델 해석 가능성을 향상시킴

3) wake-sleep 접근 방식으로 VAE를 합성하여 효율적인 부트스트래핑과 세미수퍼바이저 학습을 수행

# Model Overview

텍스트 생성에 사용 되는 VAE는 잠재변수 z로부터 조건부로 X가 생성됩니다.

바닐라 VAE는 z라는 잠재변수로 속성들을 압축하며,

이로인해 속성들은 z에서 서로 얽혀있습니다.

관심있는 속성을 해석 가능한 방식으로 제어하기 위해

얽혀있는 변수 z를

구조화 된 변수 z+c로 확장합니다.

변수 c는 문장의 두드러진 독립적, 의미론적 기능을 대상으로합니다.

우리는 문장 생성자가 결합 된 벡터 (z, c)를 조건으로하여

지정된 c라는 속성을 충족시키는 샘플을 생성하려고합니다.

그러나 비전 도메인에서도 볼 수 있듯,

- 데이터 공간에서 요소 간 거리를 계산하는 것과

- Feature 공간에서의 거리 계산을 비교하면,

후자가 특정 변수에 대한 불변성을 허용하고 더 나은 전체론적 측도를 제공합니다.

따라서 통제된 잠재변수 c의 각 속성 코드에 대해

생성 된 샘플이 원하는 속성과 얼마나 잘 일치 하는지를 측정하기 위해

Discriminator를 설정하고

개선 된 결과를 생성하도록 Generator를 학습시킵니다.

![](./img/4.png)

Discriminator를 적용하기 어렵게 만드는 점이 있습니다.

텍스트 샘플이 SGD를 적용하기에는 불연속적이고 이산적이라는 것입니다.

따라서 softmax방식을 이용해 학습을 시도하며,

이는 간단하면서도 효과적인(덜 튀고 수렴이 빠른)방식 입니다.

![](./img/3.png)

Discriminators는 속성과 대응되는 레이블에 맞게 학습됩니다.

즉, 생성기와 판별 기는 한 쌍의 공동 학습자를 형성하고 서로에게 피드백 신호를 제공합니다.

이 과정은 wake-sleep 알고리즘과 유사합니다.

우리는 VAE / wake-sleep 학습을 결합하여

매우 효율적인 세미-수퍼바이저 프레임 워크를 구현할 수 있음을 보여 주고자 합니다.

# Model Structure

우리의 Generator는 LSTM-RNN입니다.

통상적으로 텍스트 분석은 Bi-Directional + Attention + LSTM이 최고라고 합니다.

![](./img/5.png)

G는 x_hat을 생성하며 z와 c를 조건부로 합니다.

x_hat<t는 t시점 이전의 토큰들을 의미합니다.

![](./img/6.png)

o_t는 로짓백터입니다.

타우는 temperature라는데 이게 뭔가요?

![](./img/7.png)

![](./img/8.png)

VAE의 인코더에서 z로의 차원축소를 할 경우

z의 Prior는 가우시안 분포를 따른다고 가정해줍니다.

결과적으로 z는 연속형 변수입니다.

하지만 그에 더해지는 c는 이산형이어도 되고 연속형이어도 됩니다. (속성 구분만 가능하다면)

![](./img/9.png)

세타G는 Generator의 파라미터이고 세타E는 인코더의 파라미터입니다.

q_D(c|x)는 그냥 Discriminator에 의해 만들어진 확률값입니다.

### 원래 VAE

![](./img/10.png)
![](./img/11.png)

논문에서는 간단히 하기 위해 c로 1개의 속성만 고려합니다.

![](./img/12.png)
![](./img/13.png)
![](./img/14.png)

# Discriminator Learning

![](./img/15.png)
![](./img/16.png)
![](./img/17.png)
![](./img/18.png)
![](./img/19.png)